In [1]:
# создаём сессию Spark
from pyspark.sql import SparkSession

spark = (
    SparkSession
    .builder
    .master("local[*]")
    .config("spark.driver.memory", "4g")
    .getOrCreate()
)

In [2]:
# файл с оценками - user * item матрица
import os
import pyspark.sql.functions as sql_func

DATA_DIR = "/data/ml-latest"
ratings = (
    spark
    .read
    .csv(
        os.path.join(DATA_DIR, "ratings.csv"),
        header=True,
        inferSchema=True
    )
    .drop("timestamp")
    .cache()
)

In [3]:
%%time
(
    ratings.alias("one")
    .join(ratings.alias("two"), "userId")
    # расстояние симметрично,
    # поэтому считаем только в одну сторону
    .where("one.movieId > two.movieId")
    .groupBy("one.movieId", "two.movieId")
    .agg(
        sql_func.sum(
            sql_func.col("one.rating") *
            sql_func.col("two.rating")
        ).alias("inner_product"),
        sql_func.count("userId").alias("watched_both")
    ).select(
        sql_func.col("one.movieId").alias("movieId1"),
        sql_func.col("two.movieId").alias("movieId2"),
        sql_func.col("watched_both"),
        sql_func.col("inner_product")
    )
    .write
    .mode("overwrite")
    .parquet("/data/other/half_cooccurrences.parquet")
)

CPU times: user 4.97 s, sys: 1.36 s, total: 6.33 s
Wall time: 1h 16min 48s


Проект, посвящённый реализации популярных алгоритмов машинного обучения и анализа данных на традиционных SQL движках (Massive Parallel Computing, такие как Greenplum, Teradata, Netezza и другие):

[Apache MADlib](https://madlib.apache.org)

Вполне можно использовать и для других мощных SQL движков (HP Vertica, Oracle, PostgreSQL, PrestoDB)